In [1]:
import os
import cv2
import json

train_image_folder = '../data/bdd100k_images_100k/bdd100k/images/100k/train'
val_image_folder = '../data/bdd100k_images_100k/bdd100k/images/100k/val'
train_labels = '../data/bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_train.json'
val_labels = '../data/bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_val.json'

In [2]:
dataset = {
    'train': {
        'img_dir': '../data/bdd_yolo/train/images',
        'json_path': '../data/bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_train.json',
        'label_dir': '../data/bdd_yolo/train/labels'
    },
    'val': {
        'img_dir': '../data/bdd_yolo/val/images',
        'json_path': '../data/bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_val.json',
        'label_dir': '../data/bdd_yolo/val/labels'
    }
}

In [3]:
class_map = {
    'pedestrian': 0,
    'rider': 1,
    'car': 2,
    'truck': 3,
    'bus': 4,
    'train': 5,
    'motorcycle': 6,
    'bicycle': 7,
    'traffic light': 8,
    'traffic sign': 9
}

In [4]:
def convert_and_save(json_path, img_dir, label_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)

    os.makedirs(label_dir, exist_ok=True)

    for item in data:
        img_name = item['name']
        img_path = os.path.join(img_dir, img_name)

        if not os.path.exists(img_path):
            print(f"[!] Image not found: {img_path}")
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"[!] Failed to load image: {img_path}")
            continue

        h, w = img.shape[:2]
        lines = []

        for label in item['labels']:
            if 'box2d' not in label:
                continue
            category = label['category']
            if category not in class_map:
                continue

            box = label['box2d']
            x1, y1, x2, y2 = box['x1'], box['y1'], box['x2'], box['y2']

            x_center = (x1 + x2) / 2 / w
            y_center = (y1 + y2) / 2 / h
            bbox_width = (x2 - x1) / w
            bbox_height = (y2 - y1) / h

            class_id = class_map[category]
            lines.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

        # Write annotation .txt file
        label_path = os.path.join(label_dir, img_name.replace('.jpg', '.txt'))
        with open(label_path, 'w') as f:
            f.write('\n'.join(lines))


In [5]:
for split, paths in dataset.items():
    convert_and_save(paths['json_path'], paths['img_dir'], paths['label_dir'])